In [ ]:
# IMPORT LIBRARIES

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# GET vaccine,symptom  DATA FROM CSV FOR 2020 & 2021


tdata21=pd.read_csv("../input/covid19-vaccine-adverse-reactions/2021VAERSDATA.csv")

vax21=pd.read_csv("../input/covid19-vaccine-adverse-reactions/2021VAERSVAX.csv")

symp21=pd.read_csv("../input/covid19-vaccine-adverse-reactions/2021VAERSSYMPTOMS.csv")


##### Obtain Info on the various dataframes  

In [ ]:


print(tdata21.info(),tdata21.shape)


In [ ]:
print(vax21.info(),vax21.shape)

In [ ]:
print(symp21.info(),symp21.shape)

##### CONCAT THE DATAFRAMES INTO 1 FOR THE SYMPTOMS,VACCINE, AND OTHER DATA FOR THE YEAR 2020 AND 2021

In [ ]:
symp21.shape,tdata21.shape,vax21.shape

##### GETTING DATA FOR ONLY VACCINE RELATED TO COVID19

In [ ]:
vax21['VAX_NAME'].value_counts()

In [ ]:
vax21[(vax21["VAX_NAME"]=="COVID19 (COVID19 (MODERNA))") | (vax21["VAX_NAME"]=="COVID19 (COVID19 (PFIZER-BIONTECH))") | (vax21["VAX_NAME"]=="COVID19 (COVID19 (JANSSEN))")].info()

In [ ]:
vax_covid=vax21[(vax21["VAX_NAME"]=="COVID19 (COVID19 (MODERNA))") | (vax21["VAX_NAME"]=="COVID19 (COVID19 (PFIZER-BIONTECH))") | (vax21["VAX_NAME"]=="COVID19 (COVID19 (JANSSEN))")]

In [ ]:
vax_covid["VAX_NAME"].value_counts()

In [ ]:
symp21.shape,tdata21.shape,vax_covid.shape

##### MERGING THE 3 DATAFRAMES INTO A MASTER DATAFRAME FOR YEAR 2020/21 AND COVID19

In [ ]:
master_df=pd.merge(tdata21, pd.merge(symp21, vax_covid,on='VAERS_ID',how='inner'),on='VAERS_ID',how='right')

In [ ]:
master_df.shape

##### DIVIDING MASTER DATAFRAME INTO CATEGORICAL AND NUMERICAL FOR SEPERATE ANALYSIS


In [ ]:
df_num=master_df.select_dtypes(include=[np.number])

In [ ]:
df_cat=master_df.select_dtypes(include=[np.object])

##### NULL VALUE TREATMENT 

In [ ]:
df_cat.isnull().sum() # ALL NULLS in categorical columns 

In [ ]:
Percent = (df_cat.isnull().sum()*100/df_cat.isnull().count()).sort_values(ascending=False)
Percent

# Percent nulls in categorical columns 


In [ ]:

df_cat.drop(["ER_VISIT","BIRTH_DEFECT","X_STAY","RPT_DATE","V_FUNDBY","DISABLE","L_THREAT",
             "DATEDIED","DIED","PRIOR_VAX","HOSPITAL","OFC_VISIT","ER_ED_VISIT","SPLTTYPE"]
            ,axis=1,inplace=True) # dropping those features whose null percentage exceeds 50-60%

In [ ]:
Percent = (df_cat.isnull().sum()*100/df_cat.isnull().count()).sort_values(ascending=False)
Percent

In [ ]:
df_num.isnull().sum()# all nulls in numerical columns 

In [ ]:
Percent = (df_num.isnull().sum()*100/df_num.isnull().count()).sort_values(ascending=False)
Percent # percent nulls 

In [ ]:
df_num.drop(["CAGE_MO","HOSPDAYS"],axis=1,inplace=True)# dropping those features whose null percentage exceeds 50-60%

In [ ]:
Percent = (df_num.isnull().sum()*100/df_num.isnull().count()).sort_values(ascending=False)
Percent

In [ ]:
df_num.dropna(thresh=10,inplace=True) # drop those rows where atleast 1 null is present

In [ ]:
df_cat.dropna(thresh=26,inplace=True)# drop those rows where atleast 1 null is present


In [ ]:
df_cat.shape

In [ ]:
df_num.shape

##### DUPLICATES HANDLING 

In [ ]:
df_cat[df_cat.duplicated(keep='first')]

In [ ]:
df_cat.drop_duplicates(keep='first',inplace=True)

In [ ]:
df_cat[df_cat.duplicated(keep='first')]

In [ ]:
df_cat.shape

In [ ]:


df_num[df_num.duplicated(keep='first')]

In [ ]:
df_num.drop_duplicates(keep='first',inplace=True)

In [ ]:
df_num[df_num.duplicated(keep='first')]

In [ ]:
df_num.shape,df_cat.shape

In [ ]:
df_num.head()

##### By analysis we found there are data points having "None" and "none" misrepresented for nulls

In [ ]:
df_cat.replace(to_replace="None", value=np.nan,inplace=True)

In [ ]:
df_num.replace(to_replace="None", value=np.nan,inplace=True)

In [ ]:
df_cat.replace(to_replace="none", value=np.nan,inplace=True)

In [ ]:
df_num.replace(to_replace="none", value=np.nan,inplace=True)

In [ ]:
df_cat.replace(to_replace="NONE", value=np.nan,inplace=True)

In [ ]:
df_num.replace(to_replace="NONE", value=np.nan,inplace=True)

In [ ]:
df_cat.dropna(thresh=26,inplace=True) # removing nulls from above 

In [ ]:
df_num.dropna(thresh=10,inplace=True) # removing nulls from above 

### CATEGORICAL / NUMERICAL ANALYSIS

In [ ]:
df_cat.describe(include='all').T

In [ ]:
df_cat["CUR_ILL"].value_counts()# cHECKING FOR TEXTUAL/UNNECESSARY DATA COLUMNS FOR THOSE HAVING MORE THAN A 1000 UNIQUE CATEGORIES

In [ ]:
df_cat["LAB_DATA"].value_counts()# cHECKING FOR TEXTUAL/UNNECESSARY DATA COLUMNS FOR THOSE HAVING MORE THAN A 1000 UNIQUE CATEGORIES

In [ ]:
'''SO FROM ANALYSIS WE GET TO KNOW THAT THERE ARE A LOT OF TEXTUAL/IRRELEVANT DATA MARKED
AS CATEGORIES AVAILABLE. SO WE REMOVE FEATURES AS -

- HISTORY 
- SYMPTOM_TEXT etc..

'''

df_cat.drop(["HISTORY","SYMPTOM_TEXT","LAB_DATA","OTHER_MEDS","CUR_ILL","ALLERGIES"],axis=1,inplace=True)

In [ ]:
df_cat.describe(include='all').T

In [ ]:
df_cat.drop("TODAYS_DATE",axis=1,inplace=True)
# this feature didnot make any sense 


In [ ]:
df_cat.info()

### BIVARIATE/UNIVARIATE CATEGORICAL ANALYSIS

In [ ]:
sns.countplot(df_cat["VAX_MANU"])

In [ ]:
df_cat["V_ADMINBY"].value_counts().plot(kind='pie')

In [ ]:
pd.crosstab(df_cat["V_ADMINBY"],df_cat["VAX_NAME"]).plot(kind='bar',stacked=True)
plt.show()


'''
PRIVATE BODIES HAVE ADMINISTERED THE LARGEST AMOUNT OF VACCINATIONS
OF MODERNA/PFIZER 

WHEREAS OTHER UNDISCLOSED BODIES HAVE ADMINISTERED JANSSEN VACCINE THE MOST 
'''

In [ ]:

plt.rcParams['figure.figsize'] = (12,8)
pd.crosstab(df_cat["RECOVD"],df_cat["VAX_MANU"]).plot(kind='bar',stacked=True)
plt.show()

'''
The below graph tells us that 
Not Recovered from adverse effects of vaccine ("N") 
    - the count is max
    - people unable to recover from adverse effects are in order:  MODERNA>PFIZER>JANSSEN
Recovered from adverse effects of vaccine ("Y")
    - COUNT(U) < COUNT(Y) < COUNT(N)
    - people able to recover from adverse effects are in order: MODERNA>PFIZER>JANSSEN
    
'''


In [ ]:
pd.crosstab(df_cat["RECOVD"],df_cat["SEX"]).plot(kind='bar')
plt.show()

'''
Insight gathered from the below graph is that 
the recovery of the FEMAlE gender is the higher in comparison to any other gender 
for any of the covid vaccines .
'''

In [ ]:

'''
The most occured symptoms across
different vaccinations for covid are stated down 
below in a word cloud.

'''


from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords = set(list(STOPWORDS) + ['|']),background_color = 'white',
                      height =2000,
                      width = 2000
                     ).generate(str(df_cat["SYMPTOM1"]))
plt.rcParams['figure.figsize'] = (12,8)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Symptom 1")
plt.show()




In [ ]:
wordcloud2 = WordCloud(stopwords = set(list(STOPWORDS) + ['|']),background_color = 'white',
                      height =2000,
                      width = 2000
                     ).generate(str(df_cat["SYMPTOM2"]))
plt.rcParams['figure.figsize'] = (12,8)
plt.axis("off")
plt.imshow(wordcloud2)
plt.title("Symptom 2")
plt.show()




In [ ]:
wordcloud3 = WordCloud(stopwords = set(list(STOPWORDS) + ['|']),background_color = 'white',
                      height =2000,
                      width = 2000
                     ).generate(str(df_cat["SYMPTOM3"]))
plt.rcParams['figure.figsize'] = (12,8)
plt.axis("off")
plt.imshow(wordcloud3)
plt.title("Symptom 3")
plt.show()




In [ ]:
wordcloud4 = WordCloud(stopwords = set(list(STOPWORDS) + ['|']),background_color = 'white',
                      height =2000,
                      width = 2000
                     ).generate(str(df_cat["SYMPTOM4"]))
plt.rcParams['figure.figsize'] = (12,8)
plt.axis("off")
plt.imshow(wordcloud4)
plt.title("Symptom 4")
plt.show()


In [ ]:
wordcloud5 = WordCloud(stopwords = set(list(STOPWORDS) + ['|']),background_color = 'white',
                      height =2000,
                      width = 2000
                     ).generate(str(df_cat["SYMPTOM5"]))
plt.rcParams['figure.figsize'] = (12,8)
plt.axis("off")
plt.imshow(wordcloud5)
plt.title("Symptom 5")
plt.show()


### UNIVARIATE/BIVARIATE NUMERICAL ANALYSIS

In [ ]:
df_num.shape

In [ ]:
df_num.describe().T

In [ ]:
df_num_sc=df_num.iloc[:].apply(lambda x:(x-x.mean())/x.std()) # STANDARD SCALING the features 
df_num_sc.head()

In [ ]:
sns.distplot(df_num_sc["VAERS_ID"])

In [ ]:
sns.scatterplot(df_num_sc["AGE_YRS"],df_num_sc["NUMDAYS"])

# there seems no relation between the age of recepient and number of days for onset of symptoms after vaccine 

### UNIVARIATE/BIVARIATE CATEGORICAL-NUMERICAL ANALYSIS

In [ ]:
sns.boxplot(y=df_num["AGE_YRS"],x=df_cat["RECOVD"],hue=df_cat["SEX"])


'''


'''